In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal
import wandb
from tqdm import trange
from distance_models import SupervisedDistanceEstimator
from utils import trajectories_to_dataset
from ppo_agent import PPOAgent

In [2]:
import os
from torch.utils.tensorboard import SummaryWriter

os.environ['WANDB_API_KEY'] = 'd58c31e07030724bbab6b7e8edc93edacd934c87'


In [3]:
import wandb
import time
run_name = f"parallel_training__{int(time.time())}"
wandb.init(
    project="parallel trainin",
    entity=None,
    sync_tensorboard=True,
    name=run_name,
    monitor_gym=True,
    save_code=True,
)
writer = SummaryWriter(f"runs/{run_name}")


wandb: Currently logged in as: gamershmidt-sofya (gamershmidt-sofya-innopolis-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/home/user7/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment FetchSlide-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/user7/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment FetchSlide-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/user7/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment FetchPickAndPlace-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/user7/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment 

/home/user7/.venv/lib/python3.10/site-packages/wandb/analytics/sentry.py:258: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


In [4]:
import gymnasium as gym
import gymnasium_robotics
import numpy as np
import torch
import os

import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.env_wrappers import EnvBuilder
from src import ppo_agent, distance_models
from src.utils import trajectories_to_dataset

# Check for GPU
print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()) if torch.cuda.is_available() else "CPU")

CUDA available: True
Current device: 0
Device name: Tesla V100-SXM3-32GB


In [5]:
env_id = 'PointMaze_UMaze-v3'  # Use dense reward for initial training
train_episodes = 10000
max_episode_steps = 512
seed = 0

torch.manual_seed(seed); np.random.seed(seed)
# gym.register_envs(gymnasium_robotics)

# Initialize environment
c = 'c'; U_map = [
    [1, 1, 1, 1, 1],
    [1, c, c, c, 1],
    [1, 1, 1, c, 1],
    [1, c, c, c, 1],
    [1, 1, 1, 1, 1]
]
builder = EnvBuilder(env_id=env_id, maze_map=U_map, max_episode_steps=max_episode_steps, seed=seed)
obs_dim, act_dim = builder.get_obs_act_dim()
env = builder()
agent = ppo_agent.PPOAgent(state_dim=obs_dim, action_dim=act_dim, wandb_name="stage1", log_to_wandb=False)

In [7]:
env.reset()

(array([-1.0377516 , -0.81968546,  0.        ,  0.        ,  0.9923984 ,
        -1.1902139 ], dtype=float32),
 {'success': False})

In [6]:
def train_supervised_distance(
        env,
        agent,
        distance_model,
        *,
        batch_size: int,
        num_episodes: int,
        max_episode_steps: int,
        eval_size: int
    ):
        num_batches = (num_episodes + batch_size - 1)// batch_size
        sup_model = distance_model
        for episode in range(num_batches):
            print(episode)
            agent.train_ppo(env, num_episodes=batch_size, max_episode_steps=max_episode_steps)
            trajectories = agent.evaluate_ppo(env, num_episodes=eval_size)
            dataset = trajectories_to_dataset(trajectories)
            # sup_model = SupervisedDistanceEstimator(input_dim=4)
            sup_loss = sup_model.train_from_data(dataset, epochs=50, batch_size=16192)

In [19]:
distance_model = SupervisedDistanceEstimator(input_dim=4)

In [21]:
train_supervised_distance(env, agent, distance_model, batch_size=10, num_episodes=100, max_episode_steps=100, eval_size=10)

0


Evaluating: 100%|███████████████████████████████████████████████████| 10/10 [00:00<00:00, 10.05it/s]


Over 10 eval episodes, 4 were successful (40.0%)


Building dataset: 100%|████████████████████████████████████████████| 10/10 [00:00<00:00, 485.49it/s]


Trajectories processed : 10
Generated samples      : 272319 (train 217855 / eval 54464)


...

Over 10 eval episodes, 0 were successful (0.0%)


Building dataset: 100%|████████████████████████████████████████████| 10/10 [00:00<00:00, 478.16it/s]


Trajectories processed : 10
Generated samples      : 451500 (train 361200 / eval 90300)


Training, t_loss=0.00: 100%|████████████████████████████████████████| 50/50 [01:49<00:00,  2.19s/it]

eval_losses[0]=0.0007217505075870822 -> eval_losses[-1]=0.0004367409592055007
